# Face Detection Module

### Install Dependencies

In [ ]:
!pip install retina-face onnxruntime insightface

- RetinaFace based detection with implementation of threshold (Confidence Score + Laplacian Variance for blurry images)
- Cropping (112x112)

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
from retinaface import RetinaFace
import os

def is_blurry(image, threshold=100.0):
    """Check if image is blurry using variance of Laplacian."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    return fm < threshold, fm

def make_square_with_padding(img, target_size=None, pad_color=(0,0,0)):
    """Pad image to square shape, optionally resize."""
    h, w = img.shape[:2]
    size = max(h, w)
    square_img = np.full((size, size, 3), pad_color, dtype=np.uint8)
    y_offset = (size - h) // 2
    x_offset = (size - w) // 2
    square_img[y_offset:y_offset+h, x_offset:x_offset+w] = img
    if target_size:
        return cv2.resize(square_img, target_size, interpolation=cv2.INTER_AREA)
    return square_img

def process_image_bbox(image_path, output_dir,
                       min_score=0.99, blur_threshold=100.0,
                       margin_ratio=0.2, target_size=(112, 112)):
    """Detect faces, skip low-score & blurry ones, keep pixels intact."""
    print(f"Processing: {image_path}")
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not load image at {image_path}")
        return

    faces = RetinaFace.detect_faces(image_path)
    if not faces:
        print(f"No faces detected in {image_path}.")
        return

    h, w = img.shape[:2]

    for i, key in enumerate(faces.keys()):
        face = faces[key]
        score = face.get("score", 0.0)

        if score < min_score:
            print(f"Face {i+1} skipped (low score={score:.4f})")
            continue

        x1, y1, x2, y2 = face["facial_area"]

        bw, bh = x2 - x1, y2 - y1
        margin_x = int(bw * margin_ratio)
        margin_y = int(bh * margin_ratio)
        x1 = max(0, x1 - margin_x)
        y1 = max(0, y1 - margin_y)
        x2 = min(w, x2 + margin_x)
        y2 = min(h, y2 + margin_y)

        crop_w, crop_h = x2 - x1, y2 - y1
        if crop_w > crop_h:
            diff = crop_w - crop_h
            expand_top = diff // 2
            expand_bottom = diff - expand_top
            if y1 - expand_top >= 0 and y2 + expand_bottom <= h:
                y1 -= expand_top
                y2 += expand_bottom
            else:
                x1 += diff // 2
                x2 -= diff - diff // 2
        elif crop_h > crop_w:
            diff = crop_h - crop_w
            expand_left = diff // 2
            expand_right = diff - expand_left
            if x1 - expand_left >= 0 and x2 + expand_right <= w:
                x1 -= expand_left
                x2 += expand_right
            else:
                y1 += diff // 2
                y2 -= diff - diff // 2

        x1, x2 = max(0, x1), min(w, x2)
        y1, y2 = max(0, y1), min(h, y2)

        cropped_face = img[y1:y2, x1:x2]

        blurry, fm = is_blurry(cropped_face, blur_threshold)
        if blurry:
            print(f"Face {i+1} skipped (blurry, LapVar={fm:.2f})")
            continue

        final_face = make_square_with_padding(cropped_face, target_size=target_size)

        cv2_imshow(final_face)

        save_path = os.path.join(
            output_dir,
            f'{os.path.splitext(os.path.basename(image_path))[0]}_TN_{i+1}.jpg'
        )
        cv2.imwrite(save_path, final_face)
        print(f"Saved face to: {save_path} (score={score:.4f}, LapVar={fm:.2f})\n")

def main(images_dir='/content/images', output_dir='/content/detected_faces'):
    os.makedirs(output_dir, exist_ok=True)
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')

    print(f"Total number of images found for detection : {len(os.listdir(images_dir))}")
    for filename in os.listdir(images_dir):
        if filename.lower().endswith(valid_extensions):
            image_path = os.path.join(images_dir, filename)
            process_image_bbox(image_path, output_dir)

if __name__ == '__main__':
    main()

In [ ]:
print(f"Finished processing. From {len(os.listdir('/content/images'))} images, the face detection system detected {len(os.listdir('/content/detected_faces'))} faces.")

# Face Embedding Generation

### Install dependencies first if needed

In [ ]:
!pip install keras-facenet umap-learn scikit-learn matplotlib opencv-python tensorflow numpy

### FaceNet

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras_facenet import FaceNet
from sklearn.cluster import DBSCAN
import umap
import cv2

# Load images from a folder
image_folder = "/content/detected_faces"
image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder)
               if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

images = []

# Images Preprocessing (160x160 for FaceNet)
for path in image_paths:
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (160, 160))
    images.append(img)
images = np.array(images)

print(f"Loaded {len(images)} images.")

# Extract embeddings
embedder = FaceNet()
embeddings = embedder.embeddings(images)
print("Embeddings shape:", embeddings.shape)

### Optional Dimentionality Reduction
based on model requirements

In [ ]:
# Dimensionality reduction (UMAP)
reducer = umap.UMAP(n_neighbors=15, min_dist=0.0, metric='cosine', random_state=42)
embeddings_2d = reducer.fit_transform(embeddings)
print("Reduced embeddings shape:", embeddings_2d.shape)

# Face Clustering Module

### DBSCAN (eps=0.35, min_samples=3)

In [ ]:
# Clustering (DBSCAN)
dbscan_eps = 0.35
dbscan_min_samples = 3

clustering = DBSCAN(eps=dbscan_eps, min_samples=dbscan_min_samples, metric="cosine")
labels = clustering.fit_predict(embeddings)
print("Initial clusters:", set(labels))

num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f"Found {num_clusters} clusters (DBSCAN).")

# Visualization
plt.figure(figsize=(10, 8))
scatter = plt.scatter(embeddings_2d[:,0], embeddings_2d[:,1], c=labels, cmap='tab20')
plt.legend(*scatter.legend_elements(), title="Clusters")
plt.title("Face Clusters (UMAP projection)")
plt.show()

# Display faces by cluster
import math

def show_cluster_faces(images, labels):
    unique_labels = set(labels)
    for label in unique_labels:
        if label == -1:
            print("Displaying noise/outliers:")
        else:
            print(f"Displaying cluster {label}:")
        cluster_indices = [i for i, l in enumerate(labels) if l == label]
        cluster_images = [images[i] for i in cluster_indices]

        cols = 5
        rows = math.ceil(len(cluster_images) / cols)
        plt.figure(figsize=(15, 3*rows))
        for i, img in enumerate(cluster_images):
            plt.subplot(rows, cols, i+1)
            plt.imshow(img)
            plt.axis('off')
        plt.show()

show_cluster_faces(images, labels)

### DBSCAN (eps=0.25, min_samples=2)

In [ ]:
# -------------------------------
# Clustering (DBSCAN)
# -------------------------------
clustering = DBSCAN(eps=0.25, min_samples=2, metric='cosine')
labels = clustering.fit_predict(embeddings)

num_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f"Found {num_clusters} clusters (DBSCAN).")

# -------------------------------
# Visualization
# -------------------------------
plt.figure(figsize=(10, 8))
scatter = plt.scatter(embeddings_2d[:,0], embeddings_2d[:,1], c=labels, cmap='tab20')
plt.legend(*scatter.legend_elements(), title="Clusters")
plt.title("Face Clusters (UMAP projection)")
plt.show()

# -------------------------------
# Display faces by cluster
# -------------------------------
import math

def show_cluster_faces(images, labels):
    unique_labels = set(labels)
    for label in unique_labels:
        if label == -1:
            print("Displaying noise/outliers:")
        else:
            print(f"Displaying cluster {label}:")
        cluster_indices = [i for i, l in enumerate(labels) if l == label]
        cluster_images = [images[i] for i in cluster_indices]

        cols = 5
        rows = math.ceil(len(cluster_images) / cols)
        plt.figure(figsize=(15, 3*rows))
        for i, img in enumerate(cluster_images):
            plt.subplot(rows, cols, i+1)
            plt.imshow(img)
            plt.axis('off')
        plt.show()

show_cluster_faces(images, labels)

# Re-Clustering

In [ ]:
import os
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
import umap
from retinaface import RetinaFace
from keras_facenet import FaceNet
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def check_new_images(images_dir, faces_dir):
    """Compare processed images with raw images and return new ones."""
    processed = set([f.split("_TN_")[0] for f in os.listdir(faces_dir)])
    originals = set([os.path.splitext(f)[0] for f in os.listdir(images_dir)
                     if f.lower().endswith(('.jpg','.jpeg','.png'))])
    new_images = [f for f in originals if f not in processed]
    return [os.path.join(images_dir, f) for f in os.listdir(images_dir)
            if os.path.splitext(f)[0] in new_images]

In [ ]:
#  Embeddings
def get_embeddings(face_paths, embedder):
    faces = []
    for path in face_paths:
        img = cv2.imread(path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (160,160))
        faces.append(img)
    if len(faces) == 0:
        return np.array([])
    return embedder.embeddings(np.array(faces))

In [ ]:
#  Reclustering
def recluster_embeddings(embeddings, labels, similarity_threshold=0.8,
                         eps=0.35, min_samples=3):
    """Assign unlabeled faces (-1) to closest cluster or new DBSCAN cluster"""
    labeled_indices = np.where(labels != -1)[0]
    unlabeled_indices = np.where(labels == -1)[0]

    if len(unlabeled_indices) == 0:
        return labels

    labeled_embeddings = embeddings[labeled_indices]
    labeled_labels = labels[labeled_indices]
    unlabeled_embeddings = embeddings[unlabeled_indices]

    sim_matrix = cosine_similarity(unlabeled_embeddings, labeled_embeddings)
    best_match_idx = np.argmax(sim_matrix, axis=1)
    best_match_sim = np.max(sim_matrix, axis=1)

    new_labels = labels.copy()
    for i, (idx, sim) in enumerate(zip(unlabeled_indices, best_match_sim)):
        if sim > similarity_threshold:
            new_labels[idx] = labeled_labels[best_match_idx[i]]

    remaining_unlabeled = np.where(new_labels == -1)[0]
    if len(remaining_unlabeled) > 0:
        reclust = DBSCAN(eps=eps, min_samples=min_samples, metric="cosine")
        reclust_labels = reclust.fit_predict(embeddings[remaining_unlabeled])

        max_label = np.max(labels) if len(labeled_indices) > 0 else -1
        reclust_labels[reclust_labels != -1] += max_label + 1

        for i, idx in enumerate(remaining_unlabeled):
            new_labels[idx] = reclust_labels[i]

    return new_labels

In [ ]:
#  Full Pipeline
def process_pipeline(images_dir="/content/images",
                     faces_dir="/content/detected_faces"):
    os.makedirs(faces_dir, exist_ok=True)

    embedder = FaceNet()

    new_images = check_new_images(images_dir, faces_dir)
    print(f"📸 Found {len(new_images)} new images to process")
    new_faces = []

    new_faces = []
    for img_path in new_images:
        new_faces.extend(process_image_bbox(img_path, faces_dir))

    face_files = [os.path.join(faces_dir,f) for f in os.listdir(faces_dir)
                  if f.lower().endswith(('.jpg','.jpeg','.png'))]
    embeddings = get_embeddings(face_files, embedder)

    if embeddings.size == 0:
        print("⚠️ No faces found, skipping clustering")
        return [], []

    labels_path = os.path.join(faces_dir,"labels.npy")
    if not os.path.exists(labels_path):
        clustering = DBSCAN(eps=0.35, min_samples=3, metric="cosine")
        labels = clustering.fit_predict(embeddings)
    else:
        old_labels = np.load(labels_path)
        if len(old_labels) != len(face_files):
            diff = len(face_files) - len(old_labels)
            labels = np.concatenate([old_labels, -1*np.ones(diff, dtype=int)])
        else:
            labels = old_labels
        labels = recluster_embeddings(embeddings, labels)

    np.save(labels_path, labels)
    print(f"✅ Processed {len(face_files)} faces into {len(set(labels)) - (1 if -1 in labels else 0)} clusters.")

    return labels, face_files

In [ ]:
#  Visualization
def show_cluster_faces(images, labels, max_images=20):
    unique_labels = sorted(set(labels))

    for label in unique_labels:
        cluster_indices = [i for i, l in enumerate(labels) if l == label]
        cluster_images = [images[i] for i in cluster_indices]

        if label == -1:
            print(f"Displaying noise/outliers: {len(cluster_images)} images")
        else:
            print(f"Displaying cluster {label}: {len(cluster_images)} images")
            if len(cluster_images) > max_images:
                print(f"  (Showing first {max_images} of {len(cluster_images)} images)")
                cluster_images = cluster_images[:max_images]

        cols = 5
        rows = math.ceil(len(cluster_images) / cols)
        plt.figure(figsize=(15, 3*rows))
        for i, img in enumerate(cluster_images):
            plt.subplot(rows, cols, i+1)
            plt.imshow(img)
            plt.axis('off')
            if label == -1:
                plt.title(f"Noise #{cluster_indices[i]}")
        plt.show()

In [ ]:
print(f'Images in images folder: {len(os.listdir("/content/images"))}')

In [ ]:
def process_image_bbox(image_path, output_dir,
                       min_score=0.99, blur_threshold=100.0,
                       margin_ratio=0.2, target_size=(112, 112)):
    """Detect faces, skip low-score & blurry ones, keep pixels intact."""
    saved_paths = []
    print(f"Processing: {image_path}")
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not load image at {image_path}")
        return

    faces = RetinaFace.detect_faces(image_path)
    if not faces:
        print(f"No faces detected in {image_path}.")
        return

    h, w = img.shape[:2]

    for i, key in enumerate(faces.keys()):
        face = faces[key]
        score = face.get("score", 0.0)

        if score < min_score:
            print(f"Face {i+1} skipped (low score={score:.4f})")
            continue

        x1, y1, x2, y2 = face["facial_area"]

        bw, bh = x2 - x1, y2 - y1
        margin_x = int(bw * margin_ratio)
        margin_y = int(bh * margin_ratio)
        x1 = max(0, x1 - margin_x)
        y1 = max(0, y1 - margin_y)
        x2 = min(w, x2 + margin_x)
        y2 = min(h, y2 + margin_y)

        crop_w, crop_h = x2 - x1, y2 - y1
        if crop_w > crop_h:
            diff = crop_w - crop_h
            expand_top = diff // 2
            expand_bottom = diff - expand_top
            if y1 - expand_top >= 0 and y2 + expand_bottom <= h:
                y1 -= expand_top
                y2 += expand_bottom
            else:
                x1 += diff // 2
                x2 -= diff - diff // 2
        elif crop_h > crop_w:
            diff = crop_h - crop_w
            expand_left = diff // 2
            expand_right = diff - expand_left
            if x1 - expand_left >= 0 and x2 + expand_right <= w:
                x1 -= expand_left
                x2 += expand_right
            else:
                y1 += diff // 2
                y2 -= diff - diff // 2

        x1, x2 = max(0, x1), min(w, x2)
        y1, y2 = max(0, y1), min(h, y2)

        cropped_face = img[y1:y2, x1:x2]

        blurry, fm = is_blurry(cropped_face, blur_threshold)
        if blurry:
            print(f"Face {i+1} skipped (blurry, LapVar={fm:.2f})")
            continue

        final_face = make_square_with_padding(cropped_face, target_size=target_size)

        cv2_imshow(final_face)

        save_path = os.path.join(
            output_dir,
            f'{os.path.splitext(os.path.basename(image_path))[0]}_TN_{i+1}.jpg'
        )
        cv2.imwrite(save_path, final_face)
        print(f"Saved face to: {save_path} (score={score:.4f}, LapVar={fm:.2f})\n")
        saved_paths.append(save_path)

    return saved_paths

In [ ]:
labels, face_files = process_pipeline()

In [ ]:
labels, face_files = process_pipeline()

In [ ]:
# Final cluster information
final_clusters = set(labels) - {-1}
print(f"Final clusters: {len(final_clusters)}")

In [ ]:
# Dimensionality reduction for visualization
reducer = umap.UMAP(n_components=2, metric='cosine', random_state=42)
embeddings = get_embeddings(face_files, FaceNet())
embeddings_2d = reducer.fit_transform(embeddings)

In [ ]:
plt.figure(figsize=(12, 10))
scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=labels, cmap='tab20', s=10)
plt.colorbar(scatter, label="Cluster ID")
plt.title("Final Face Clusters (UMAP projection)")
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")

unique_labels = sorted(set(labels))
for label in unique_labels:
    if label == -1:
        plt.scatter([], [], c='gray', label=f'Noise ({-1})')
    else:
        plt.scatter([], [], c=scatter.cmap(scatter.norm(label)), label=f'Cluster {label}')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

### Visualize reclustered results

In [ ]:
# Display faces by cluster
face_imgs = [cv2.cvtColor(cv2.imread(f), cv2.COLOR_BGR2RGB) for f in face_files]
show_cluster_faces(face_imgs, labels)

# Move Images of Individual in one folder

In [ ]:
import shutil

def save_faces_by_cluster(face_files, labels, faces_dir):
    unique_labels = set(labels)
    for lbl in unique_labels:
        cluster_name = f"person_{lbl}" if lbl != -1 else "outliers"
        cluster_dir = os.path.join(faces_dir, cluster_name)
        os.makedirs(cluster_dir, exist_ok=True)

        indices = [i for i, l in enumerate(labels) if l == lbl]
        for i in indices:
            src = face_files[i]
            dst = os.path.join(cluster_dir, os.path.basename(src))

            if not os.path.exists(dst):
                shutil.copy(src, dst)

In [ ]:
save_faces_by_cluster(face_files, labels, '/content/individuals_recognized')